In [11]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import tweepy
import io
import os
from dotenv import find_dotenv, load_dotenv
import time

In [12]:
df_tae = pd.read_csv('twitter-archive-enhanced.csv')

df_tae.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
450,818646164899774465,NaN,NaN,2017-01-10 02:30:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Seamus. He's very bad a...,8.083449e+17,4.196984e+09,2016-12-12 16:16:49 +0000,"https://vine.co/v/5QWd3LZqXxd,https://vine.co/...",11,10,Seamus,NaN,NaN,NaN,NaN
894,759197388317847553,NaN,NaN,2016-07-30 01:22:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Luna. She's just heckin precious af I ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/759197388...,12,10,Luna,NaN,NaN,NaN,NaN
487,814153002265309185,NaN,NaN,2016-12-28 16:56:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Oshie. He's ready to party. Bought tha...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/814153002...,12,10,Oshie,NaN,NaN,NaN,NaN
2001,672482722825261057,NaN,NaN,2015-12-03 18:29:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is light saber pup. Ready to fight off ev...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/672482722...,10,10,light,NaN,NaN,NaN,NaN
2184,668988183816871936,NaN,NaN,2015-11-24 03:03:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Honor to rate this dog. Lots of fur on him. Tw...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668988183...,7,10,NaN,NaN,NaN,NaN,NaN


In [13]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

page = requests.get(url)
df_img = pd.read_csv(io.StringIO(page.content.decode('utf-8')), sep='\t')

with open('image-predictions.tsv', 'wb') as file:
    file.write(page.content)

df_img.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
610,680100725817409536,https://pbs.twimg.com/media/CW-loUBWYAAn2Cb.jpg,1,golden_retriever,0.698961,True,chow,0.145971,True,Pomeranian,0.034888,True
1122,726935089318363137,https://pbs.twimg.com/media/ChaXmuAXEAE66KP.jpg,2,teddy,0.821615,False,toy_poodle,0.083749,True,Lakeland_terrier,0.033318,True
432,674291837063053312,https://pbs.twimg.com/media/CVuQ2LeUsAAIe3s.jpg,1,Cardigan,0.611525,True,Pembroke,0.368566,True,Chihuahua,0.003330,True
287,671154572044468225,https://pbs.twimg.com/media/CVBrhXoWIAAox_C.jpg,1,Labrador_retriever,0.495047,True,Chesapeake_Bay_retriever,0.350188,True,golden_retriever,0.142400,True
280,671115716440031232,https://pbs.twimg.com/media/CVBILUgVAAA1ZUr.jpg,1,malinois,0.406341,True,kelpie,0.143366,True,dingo,0.129802,False


In [14]:
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
    
# Load credentials from environment variables
bearer_token_ = os.getenv('BEARER_TOKEN')
consumer_key_ = os.getenv('CONSUMER_KEY')
consumer_secret_ = os.getenv('CONSUMER_SECRET')
access_token_ = os.getenv('ACCESS_TOKEN')
access_secret_ = os.getenv('ACCESS_SECRET')

client = tweepy.Client(bearer_token=bearer_token_,
    consumer_key=consumer_key_,
    consumer_secret=consumer_secret_,
    access_token=access_token_,
    access_token_secret=access_secret_,
    wait_on_rate_limit=True
    )

In [ ]:
tweet_ids = df_img['tweet_id'].tolist()

# Process in batches of 100 so it works for the get_tweets method
with open('tweet_json.txt', 'a') as file:
    for i in range(0, len(tweet_ids), 100):
        try:
            tweets = client.get_tweets(tweet_ids[i:i+100], tweet_fields=['public_metrics','text'])
            if tweets.data:
                for tweet in tweets.data:
                    file.write(json.dumps(tweet.data) + '\n')
            else:
                print(f"No data returned for {tweet.id}")
                
        except tweepy.TooManyRequests as e:
            print("Rate limit reached. Retrying after 15 mins and 5 secs...")
            time.sleep(15 * 61)  # Wait 15 minutes and 5 secs.

        except Exception as e:
            print("An error occurred: " + str(e))

An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
An error occ

KeyboardInterrupt: 